In [67]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import gensim.downloader as api
import pandas as pd

In [68]:
d_model = 25
n_head = 5
total_tokens = 1193514
n_seq = 16

In [69]:
wv = api.load('glove-twitter-25')

In [70]:
input1 = np.loadtxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/input1.csv')
input2 = np.loadtxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/input2.csv')
output = np.loadtxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/output.csv')

In [71]:
input1 = input1.reshape(-1, 1, n_seq, 25)
input2 = input2.reshape(-1, 1, n_seq, 25)
OHoutput = tf.keras.utils.to_categorical(output, num_classes=total_tokens).reshape(-1,1,total_tokens)

In [72]:
n_batch, _, _, _ = input1.shape

In [73]:
def tokenize(x):
    return x.split()

def embed(x):
    embeddings = wv[x]
    return embeddings

# def posEncode(x):
#     _,_,n_seq, d_model = x.shape
#     odd_i = np.arange(0, d_model, 2, dtype='float64')
#     even_i = np.arange(1, d_model, 2, dtype='float64')
#     odd_denominator = np.power(10000, odd_i/d_model)
#     even_denominator = np.power(10000, even_i/d_model)
#     position = np.arange(0, n_seq, dtype='float64').reshape(-1,1)
#     even_PE = np.sin(position / even_denominator)
#     odd_PE  = np.cos(position / odd_denominator)
#     return x + np.ravel([even_PE.T, odd_PE.T],'F').reshape(n_seq, d_model)

def masking(x):
    mask = np.tril(np.ones((x.shape)))
    mask[mask==0] = -np.infty
    mask[mask==1] = 0
    return x + mask

In [74]:
def init_QKV(n_head, d_model):
    Q = tf.Variable(np.random.rand(1,n_head, d_model, d_model//n_head), dtype='float64')
    K = tf.Variable(np.random.rand(1,n_head, d_model, d_model//n_head), dtype='float64')
    V = tf.Variable(np.random.rand(1,n_head, d_model, d_model//n_head), dtype='float64')
    return Q, K, V

def init_GB(n_batch, d_model):
    gamma = tf.Variable(np.ones((n_batch, 1, 1,d_model)), dtype='float64')
    beta = tf.Variable(np.zeros((n_batch, 1, 1,d_model)), dtype='float64')
    return gamma, beta

def init_WB(input_size, neurons):
    w = tf.Variable(np.random.rand(1,1, input_size, neurons), dtype= 'float64')
    b = tf.Variable(np.zeros((n_batch, 1, 1, neurons)) , dtype= 'float64')
    return w, b

In [75]:
def context(input, Q,K,V, cross_input=None, mask=False):    
    if (cross_input==None):
        cross_input = input
    raw_attention = ( cross_input @ Q ) @ tf.transpose( input @ K, perm=(0,1,3,2) )
    if (mask == True):
        raw_attention = masking( raw_attention )
    score = tf.nn.softmax( raw_attention / (d_model) ** .5 )
    context = score @ ( input @ V )
    
    return context

def concat4D(x):
    a,b,c,d = x.shape 
    x1 = tf.transpose(x, perm=(0,1,3,2))
    x2 = tf.reshape(x1, [a, 1, b*d, c])
    x3 = tf.transpose(x2, perm=(0,1,3,2))
    return x3

In [76]:
def add_norm(context, prev_input, gamma, beta):
    
    context = context + prev_input
    a,_,b,_ = context.shape
    #________Mean__________
    m = tf.reduce_mean(context, axis=3)
    mean = tf.transpose(tf.reshape(m, [a,1,1,b]), perm=(0,1,3,2))
    #________Sigma__________
    s = tf.math.reduce_std(context, axis=3)
    sigma = tf.transpose(tf.reshape(s, [a,1,1,b]), perm=(0,1,3,2))

    context = (context - mean) / sigma
    context = context * gamma + beta
    
    return context

def feed_forward(x, w, b):
    z = x @ w + b
    a = tf.nn.relu(z)
    return a

TRAINING || Assume no positional encode

In [77]:
Q1, K1, V1 = init_QKV(n_head=n_head, d_model=d_model)
gamma1, beta1 = init_GB(n_batch=n_batch, d_model=d_model)
gamma2, beta2 = init_GB(n_batch=n_batch, d_model=d_model)
w1, b1 = init_WB(d_model, 16)
w2, b2 = init_WB(16, d_model)

In [78]:
context1 = concat4D(context(input1,Q1, K1, V1))
input1_1 = add_norm(context1, input1, gamma1, beta1)

In [79]:
A1 = feed_forward(input1_1, w1, b1)
A2 = feed_forward(A1, w2, b2)

In [80]:
input1_2 = add_norm(A2, input1_1, gamma2, beta2)

DECODER

In [81]:
Q2, K2, V2 = init_QKV(n_head=n_head, d_model=d_model)
Q_d, K_e, V_e = init_QKV(n_head=n_head, d_model=d_model)
gamma3, beta3 = init_GB(n_batch=n_batch, d_model=d_model)
gamma4, beta4 = init_GB(n_batch=n_batch, d_model=d_model)
gamma5, beta5 = init_GB(n_batch=n_batch, d_model=d_model)
w3, b3 = init_WB(d_model, 16)
w4, b4 = init_WB(16, d_model)

In [82]:
context2 = concat4D(context(input2,Q2, K2, V2, mask=True))
input2_1 = add_norm(context2, input2, gamma3, beta3)

In [83]:
cross_context = concat4D(context(input1_2,Q2, K2, V2, cross_input=input2_1))
input2_2 = add_norm(cross_context, input2_1, gamma4, beta4)

In [84]:
A3 = feed_forward(input2_2, w3, b3)
A4 = feed_forward(A3, w4, b4)

In [85]:
input2_3 = add_norm(A4, input2_2, gamma5, beta5)

Linear

In [86]:
W = tf.Variable(np.random.rand(1, 25, total_tokens))

In [87]:
predictions = tf.nn.softmax(input2_3[:, :, -1, :] @ W)

In [90]:
vars =          [Q1,K1,V1,
                 gamma1, beta1, gamma2, beta2,
                 w1,b1,w2,b2,
                 Q2, K2, V2, 
                 Q_d, K_e, V_e,
                 gamma3,beta3,gamma4,beta4,gamma5,beta5,
                 w3,b3,w4,b4, W]

In [91]:
for i in range(2):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(vars)

        #ENCODER
        context1 = concat4D(context(input1,Q1, K1, V1))
        input1_1 = add_norm(context1, input1, gamma1, beta1)
        A1 = feed_forward(input1_1, w1, b1)
        A2 = feed_forward(A1, w2, b2)
        input1_2 = add_norm(A2, input1_1, gamma2, beta2)
        #DECODER
        context2 = concat4D(context(input2,Q2, K2, V2, mask=True))
        input2_1 = add_norm(context2, input2, gamma3, beta3)
        cross_context = concat4D(context(input1_2,Q2, K2, V2, cross_input=input2_1))
        input2_2 = add_norm(cross_context, input2_1, gamma4, beta4)
        A3 = feed_forward(input2_2, w3, b3)
        A4 = feed_forward(A3, w4, b4)
        input2_3 = add_norm(A4, input2_2, gamma5, beta5)
        predictions = tf.nn.softmax(input2_3[:, :, -1, :] @ W)
        loss = tf.reduce_mean(tf.square(predictions - OHoutput))